In [1]:
from datasets import load_dataset
import pandas as pd

/home/jaykold/dev/llm-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#full_data = load_dataset("squad_v2", split="train")

In [3]:
# shuffled_data = full_data.shuffle(seed=42)

# type(shuffled_data)

In [4]:
# shuffled_data = pd.DataFrame(shuffled_data)

In [5]:
# shuffled_data.head()

In [ ]:
#shuffled_data.to_csv("data/squad_v2.csv")
# data = shuffled_data.head(600)

In [ ]:
# df = pd.DataFrame(data)
# df['title']

0                     Canon_law
1      Athanasius_of_Alexandria
2         Jehovah%27s_Witnesses
3                       Estonia
4               Classical_music
                 ...           
595     Northwestern_University
596                       Paper
597                    Zhejiang
598    Brigham_Young_University
599                     YouTube
Name: title, Length: 600, dtype: object

In [ ]:
# df.to_json("data/squad_v2_600.json", orient="records", indent=2)

In [6]:
import json

with open("data/squad_v2_600.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [7]:
data[11]

{'id': '57325c2fe99e3014001e6716',
 'title': 'Jehovah%27s_Witnesses',
 'context': 'Sociologist Rodney Stark states that Jehovah\'s Witness leaders are "not always very democratic" and that members "are expected to conform to rather strict standards," but adds that "enforcement tends to be very informal, sustained by the close bonds of friendship within the group", and that Jehovah\'s Witnesses see themselves as "part of the power structure rather than subject to it." Sociologist Andrew Holden states that most members who join millenarian movements such as Jehovah\'s Witnesses have made an informed choice. However, he also states that defectors "are seldom allowed a dignified exit", and describes the administration as autocratic.',
 'question': "How do Jehovah's Witnesses see themselves in regards to the power structure of the church?",
 'answers': {'text': ['"part of the power structure rather than subject to it."'],
  'answer_start': [330]}}

In [8]:
from urllib.parse import unquote

def clean_text(value):
    """Decode unicode escapes and URL encodings in a string."""
    if not isinstance(value, str):
        return value
    try:
        # Decode URL-encoded characters (%20, %27, etc.)
        value = unquote(value)
        # Decode unicode escapes (\u00f3 etc.)
        #value = value.encode('utf-8').decode('unicode_escape')
        # Decode Python-style backslash escapes (e.g., \' -> ')
        #value = value.encode('utf-8').decode('unicode_escape')
        # Replace underscores with spaces in certain cases
        value = value.replace('_', ' ')
        return value
    except Exception:
        return value  # Fail silently if decoding fails

def clean_data(obj):
    """Recursively clean all strings in a nested structure."""
    if isinstance(obj, dict):
        return {k: clean_data(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [clean_data(item) for item in obj]
    elif isinstance(obj, str):
        return clean_text(obj)
    else:
        return obj


In [9]:
cleaned_data = clean_data(data)

In [15]:
cleaned_data[11]['answers']['text'][0].replace('"', "")

'part of the power structure rather than subject to it.'

In [16]:
documents = []

for item in cleaned_data:
    if not item['answers']['text']:
        continue

    documents.append({
        "id": item["id"],
        'title': item['title'],
        "context": item['context'].replace("\n", " "),
        "question": item['question'].replace("\n", " "),
        "answer": item['answers']['text'][0].replace('"', "")
    })

In [17]:
documents[10]

{'id': '57325c2fe99e3014001e6716',
 'title': "Jehovah's Witnesses",
 'context': 'Sociologist Rodney Stark states that Jehovah\'s Witness leaders are "not always very democratic" and that members "are expected to conform to rather strict standards," but adds that "enforcement tends to be very informal, sustained by the close bonds of friendship within the group", and that Jehovah\'s Witnesses see themselves as "part of the power structure rather than subject to it." Sociologist Andrew Holden states that most members who join millenarian movements such as Jehovah\'s Witnesses have made an informed choice. However, he also states that defectors "are seldom allowed a dignified exit", and describes the administration as autocratic.',
 'question': "How do Jehovah's Witnesses see themselves in regards to the power structure of the church?",
 'answer': 'part of the power structure rather than subject to it.'}

##### Hybrid Search

In [20]:
from qdrant_client import models, QdrantClient
from fastembed import TextEmbedding

2025-10-14 15:01:26.341257297 [W:onnxruntime:Default, device_discovery.cc:164 DiscoverDevicesForPlatform] GPU device discovery failed: device_discovery.cc:89 ReadFileContents Failed to open file: "/sys/class/drm/card0/device/vendor"


In [28]:
#client = QdrantClient("localhost", port=6333)

In [ ]:
ENBEDDING_DIMENSIONALITY=512

for model in TextEmbedding.list_supported_models():
    if model['dim'] == ENBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

{
  "model": "BAAI/bge-small-zh-v1.5",
  "sources": {
    "hf": "Qdrant/bge-small-zh-v1.5",
    "url": "https://storage.googleapis.com/qdrant-fastembed/fast-bge-small-zh-v1.5.tar.gz",
    "_deprecated_tar_struct": true
  },
  "model_file": "model_optimized.onnx",
  "description": "Text embeddings, Unimodal (text), Chinese, 512 input tokens truncation, Prefixes for queries/documents: not so necessary, 2023 year.",
  "license": "mit",
  "size_in_GB": 0.09,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "Qdrant/clip-ViT-B-32-text",
  "sources": {
    "hf": "Qdrant/clip-ViT-B-32-text",
    "url": null,
    "_deprecated_tar_struct": false
  },
  "model_file": "model.onnx",
  "description": "Text embeddings, Multimodal (text&image), English, 77 input tokens truncation, Prefixes for queries/documents: not necessary, 2021 year",
  "license": "mit",
  "size_in_GB": 0.25,
  "additional_files": [],
  "dim": 512,
  "tasks": {}
}
{
  "model": "jinaai/jina-embeddings-v2-small-e

In [27]:
embedding_model="jinaai/jina-embeddings-v2-small-en"

In [ ]:
collection_name="vector_rag"

# Create a collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=ENBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)